In [ ]:
import pickle
import numpy as np
import torch
from tqdm import tqdm
import numqi
import matplotlib.pyplot as plt

In [ ]:
train_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def save_data(xdata, ydata, filepath='data_boundary.pkl'):
    with open(filepath, 'wb') as fid:
        pickle.dump(dict(xdata=xdata, ydata=ydata), fid)

def load_data(filepath='data_boundary.pkl'):
    with open(filepath, 'rb') as fid:
        tmp0 = pickle.load(fid)
        ret = tmp0['xdata'], tmp0['ydata']
    return ret

In [ ]:
dim = 3
n = 1000000
num_sample = n * dim
num_list = [n] * dim
batch_size = 4096*4
num_workers = 2

In [ ]:
xdata = []
ydata = []
for i in range(len(num_list)):
    for _ in tqdm(range(num_list[i]//2)):
        rho = numqi.random.rand_density_matrix(dim, k=i+1)
        vec = numqi.gellmann.dm_to_gellmann_basis(rho)
        # normalize
        vec = vec / np.linalg.norm(vec)
        beta_dm_l, beta_dm_u = numqi.entangle.get_density_matrix_boundary(rho)
        xdata.append(vec)
        ydata.append(beta_dm_u)
        xdata.append(-vec)
        ydata.append(-beta_dm_l)

xdata = np.array(xdata)
ydata = np.array(ydata)
print(f'min: {np.min(ydata)}, max: {np.max(ydata)}, mean: {np.mean(ydata)}, variance: {np.var(ydata)}')
save_data(xdata, ydata, f'data_dm_boundary_RankSampling_{dim}_{num_sample}.pkl') 

In [ ]:
# check the real boundary
print(np.sqrt(1/(2*dim*(dim-1))))
print(np.sqrt((dim-1)/(2*dim)))

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, dim, device='cpu'):
        super(Net, self).__init__()
        tmp0 = [dim**2-1, 128, 256, 256, 512, 512, 512, 256, 256, 128, 1]
        self.fc_list = torch.nn.ModuleList([torch.nn.Linear(tmp0[x], tmp0[x+1]) for x in range(len(tmp0)-1)])
        self.bn_list = torch.nn.ModuleList([torch.nn.BatchNorm1d(tmp0[x+1]) for x in range(len(tmp0)-1)])
        self.a = torch.sqrt(torch.tensor(1/(2*dim*(dim-1)), device=device))
        self.b = torch.sqrt(torch.tensor((dim-1)/(2*dim), device=device))
        self.scale = torch.tensor(0.05, device=device)

    def forward(self, x):
        for ind0 in range(len(self.fc_list)):
            y = self.fc_list[ind0](x)
            y = self.bn_list[ind0](y)
            if ind0<(len(self.fc_list)-1):
                y = torch.nn.functional.leaky_relu(y)
            if x.shape[-1]==y.shape[-1]:
                x = y + x
            else:
                x = y
        x = (1 + self.scale) * (self.b - self.a) * torch.sigmoid(x) + self.a * (1 - self.scale)
        x = x[:,0] 
        return x

In [ ]:
xdata, ydata = load_data(f'data_dm_boundary_RankSampling_{dim}_{num_sample}.pkl')
tmp0 = torch.tensor(xdata, dtype=torch.float32)
tmp1 = torch.tensor(ydata, dtype=torch.float32)
dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(tmp0, tmp1), batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
model = Net(dim, device=train_device).to(train_device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.7)
loss_history = []
model.train()
for epoch in range(10):
    with tqdm(dataloader) as pbar:
        for x, y in pbar:
            x, y = x.to(train_device), y.to(train_device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = torch.nn.functional.mse_loss(y_pred, y)
            loss.backward()
            optimizer.step()
            loss_history.append(loss.item())
            pbar.set_postfix(train_loss='{:.5}'.format(sum(loss_history[-10:])/10))
        lr_scheduler.step()

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.arange(len(loss_history)), loss_history, '.')
ax.set_xlabel('iteration')
ax.set_ylabel('loss')
# log scale
ax.set_yscale('log')
ax.grid()
fig.tight_layout() 

In [ ]:
torch.save(model.state_dict(), f'model_qudit_RankSampling_{dim}_{num_sample}_{batch_size}.pt')

In [ ]:
i=0
j=62
GellMann = numqi.gellmann.all_gellmann_matrix(dim, with_I=False)
op0 = GellMann[i]
op1 = GellMann[j]

theta_list = np.linspace(0, 2*np.pi, 500)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

_, hf_plane = numqi.entangle.get_density_matrix_plane(op0, op1)
beta_dm_cs = np.array([numqi.entangle.get_density_matrix_boundary(hf_plane(x))[1] for x in theta_list])
nr_dm_cs = beta_dm_cs.reshape(-1,1) * direction

vec_list = np.array([numqi.gellmann.dm_to_gellmann_basis(hf_plane(x)) for x in theta_list])
model = Net(dim)
device = torch.device('cpu')
model.load_state_dict(torch.load(f'model_qudit_RankSampling_{dim}_{num_sample}_{batch_size}.pt', map_location=device))
model.eval()
with torch.no_grad():
    beta_dm_model = model(torch.tensor(vec_list, dtype=torch.float32)).numpy()

nr_dm_model_cs = beta_dm_model.reshape(-1,1) * direction

nr_dm_inside_ball = model.a.numpy() * direction
nr_dm_outside_ball = model.b.numpy() * direction

fig, ax = plt.subplots()
ax.plot(nr_dm_cs[:,0], nr_dm_cs[:,1], '-', color='blue')
ax.plot(nr_dm_model_cs[:,0], nr_dm_model_cs[:,1], '--', color='blue')
ax.plot(nr_dm_inside_ball[:,0], nr_dm_inside_ball[:,1], '-', color='green')
ax.plot(nr_dm_outside_ball[:,0], nr_dm_outside_ball[:,1], '-', color='green')
ax.axis('equal')
ax.set_xlabel(r'$\lambda_{%d}$'%(i+1))
ax.set_ylabel(r'$\lambda_{%d}$'%(j+1))
#fig.savefig(f'qudit{dim}_{i+1}_{j+1}.pdf')

In [ ]:
op0 = numqi.random.rand_density_matrix(dim)
op1 = numqi.random.rand_density_matrix(dim)

theta_list = np.linspace(0, 2*np.pi, 500)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

_, hf_plane = numqi.entangle.get_density_matrix_plane(op0, op1)
beta_dm_cs = np.array([numqi.entangle.get_density_matrix_boundary(hf_plane(x))[1] for x in theta_list])
nr_dm_cs = beta_dm_cs.reshape(-1,1) * direction

vec_list = np.array([numqi.gellmann.dm_to_gellmann_basis(hf_plane(x)) for x in theta_list])
model = Net(dim)
device = torch.device('cpu')
model.load_state_dict(torch.load(f'model_qudit_RankSampling_{dim}_{num_sample}_{batch_size}.pt', map_location=device))
model.eval()
with torch.no_grad():
    beta_dm_model = model(torch.tensor(vec_list, dtype=torch.float32)).numpy()

nr_dm_model_cs = beta_dm_model.reshape(-1,1) * direction

nr_dm_inside_ball = model.a.numpy() * direction
nr_dm_outside_ball = model.b.numpy() * direction

fig, ax = plt.subplots()
ax.plot(nr_dm_cs[:,0], nr_dm_cs[:,1], '-', color='blue')
ax.plot(nr_dm_model_cs[:,0], nr_dm_model_cs[:,1], '--', color='blue')
ax.plot(nr_dm_inside_ball[:,0], nr_dm_inside_ball[:,1], '-', color='green')
ax.plot(nr_dm_outside_ball[:,0], nr_dm_outside_ball[:,1], '-', color='green')
ax.axis('equal')
#fig.savefig(f'qudit{dim}_random.pdf')